# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [109]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. '''

## 2. Ask for parameters

In [123]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [124]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Decision Variables:

*  **x<sub>1A</sub>**: Liters of component C1 used in the production of product GCA.
*  **x<sub>2A</sub>**: Liters of component C2 used in the production of product GCA.
*  **x<sub>1B</sub>**: Liters of component C1 used in the production of product GCB.
*  **x<sub>2B</sub>**: Liters of component C2 used in the production of product GCB.
*  **x<sub>1C</sub>**: Liters of component C1 used in the production of product GCC.
*  **x<sub>2C</sub>**: Liters of component C2 used in the production of product GCC. 


# 2. Ask for objective

In [125]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [126]:
Markdown(response2.text)

Maximize: 

**120(x<sub>1A</sub> + x<sub>2A</sub>) + 135(x<sub>1B</sub> + x<sub>2B</sub>) + 155(x<sub>1C</sub> + x<sub>2C</sub>)** 


# 3. Ask for constraints

In [127]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [128]:
Markdown(response3.text)

Subject to:

**Availability Constraints:**

*  x<sub>1A</sub> + x<sub>1B</sub> + x<sub>1C</sub> ≤ 10,000 (Liters of C1 available)
*  x<sub>2A</sub> + x<sub>2B</sub> + x<sub>2C</sub> ≤ 15,000 (Liters of C2 available)

**Demand Constraints:**

*  x<sub>1A</sub> + x<sub>2A</sub> ≥ 6,000 (Demand for GCA)
*  x<sub>1B</sub> + x<sub>2B</sub> ≥ 7,000 (Demand for GCB)
*  x<sub>1C</sub> + x<sub>2C</sub> ≥ 9,000 (Demand for GCC)

**Critical Element Constraints:**

* 0.4x<sub>1A</sub> + 0.2x<sub>2A</sub> ≥ 0.3(x<sub>1A</sub> + x<sub>2A</sub>)  (Critical element in GCA)
* 0.4x<sub>1B</sub> + 0.2x<sub>2B</sub> ≤ 0.3(x<sub>1B</sub> + x<sub>2B</sub>)  (Critical element in GCB) 

**Component Ratio Constraint:**

* x<sub>1C</sub> ≥ 0.3x<sub>2C</sub> (Ratio of C1 to C2 in GCC)

**Non-Negativity Constraints:**

* x<sub>1A</sub>, x<sub>2A</sub>, x<sub>1B</sub>, x<sub>2B</sub>, x<sub>1C</sub>, x<sub>2C</sub> ≥ 0 


## 3. Generate the pyomo code

In [129]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [130]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x1A = pyo.Var(within=pyo.NonNegativeReals)
model.x2A = pyo.Var(within=pyo.NonNegativeReals)
model.x1B = pyo.Var(within=pyo.NonNegativeReals)
model.x2B = pyo.Var(within=pyo.NonNegativeReals)
model.x1C = pyo.Var(within=pyo.NonNegativeReals)
model.x2C = pyo.Var(within=pyo.NonNegativeReals)

# Define the objective function
model.profit = pyo.Objective(
    expr=120*(model.x1A + model.x2A) + 135*(model.x1B + model.x2B) + 155*(model.x1C + model.x2C), 
    sense=pyo.maximize
)

# Define constraints
model.availability_C1 = pyo.Constraint(expr=model.x1A + model.x1B + model.x1C <= 10000)
model.availability_C2 = pyo.Constraint(expr=model.x2A + model.x2B + model.x2C <= 15000)

model.demand_GCA = pyo.Constraint(expr=model.x1A + model.x2A >= 6000)
model.demand_GCB = pyo.Constraint(expr=model.x1B + model.x2B >= 7000)
model.demand_GCC = pyo.Constraint(expr=model.x1C + model.x2C >= 9000)

model.critical_GCA = pyo.Constraint(expr=0.4*model.x1A + 0.2*model.x2A >= 0.3*(model.x1A + model.x2A))
model.critical_GCB = pyo.Constraint(expr=0.4*model.x1B + 0.2*model.x2B <= 0.3*(model.x1B + model.x2B))

model.ratio_GCC = pyo.Constraint(expr=model.x1C >= 0.3*model.x2C)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if needed
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"x1A: {model.x1A.value:.2f} liters")
print(f"x2A: {model.x2A.value:.2f} liters")
print(f"x1B: {model.x1B.value:.2f} liters")
print(f"x2B: {model.x2B.value:.2f} liters")
print(f"x1C: {model.x1C.value:.2f} liters")
print(f"x2C: {model.x2C.value:.2f} liters")
print(f"Total Profit: ${model.profit.expr():.2f}")
```


## 4. Input problem data and try running the generated code

In [131]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x1A = pyo.Var(within=pyo.NonNegativeReals)
model.x2A = pyo.Var(within=pyo.NonNegativeReals)
model.x1B = pyo.Var(within=pyo.NonNegativeReals)
model.x2B = pyo.Var(within=pyo.NonNegativeReals)
model.x1C = pyo.Var(within=pyo.NonNegativeReals)
model.x2C = pyo.Var(within=pyo.NonNegativeReals)

# Define the objective function
model.profit = pyo.Objective(
    expr=120*(model.x1A + model.x2A) + 135*(model.x1B + model.x2B) + 155*(model.x1C + model.x2C), 
    sense=pyo.maximize
)

# Define constraints
model.availability_C1 = pyo.Constraint(expr=model.x1A + model.x1B + model.x1C <= 10000)
model.availability_C2 = pyo.Constraint(expr=model.x2A + model.x2B + model.x2C <= 15000)

model.demand_GCA = pyo.Constraint(expr=model.x1A + model.x2A >= 6000)
model.demand_GCB = pyo.Constraint(expr=model.x1B + model.x2B >= 7000)
model.demand_GCC = pyo.Constraint(expr=model.x1C + model.x2C >= 9000)

model.critical_GCA = pyo.Constraint(expr=0.4*model.x1A + 0.2*model.x2A >= 0.3*(model.x1A + model.x2A))
model.critical_GCB = pyo.Constraint(expr=0.4*model.x1B + 0.2*model.x2B <= 0.3*(model.x1B + model.x2B))

model.ratio_GCC = pyo.Constraint(expr=model.x1C >= 0.3*model.x2C)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if needed
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"x1A: {model.x1A.value:.2f} liters")
print(f"x2A: {model.x2A.value:.2f} liters")
print(f"x1B: {model.x1B.value:.2f} liters")
print(f"x2B: {model.x2B.value:.2f} liters")
print(f"x1C: {model.x1C.value:.2f} liters")
print(f"x2C: {model.x2C.value:.2f} liters")
print(f"Total Profit: ${model.profit.expr():.2f}")

Optimal Solution:
x1A: 3730.77 liters
x2A: 2269.23 liters
x1B: 3500.00 liters
x2B: 3500.00 liters
x1C: 2769.23 liters
x2C: 9230.77 liters
Total Profit: $3525000.00


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [132]:
response.text

'### Decision Variables:\n\n*  **x<sub>1A</sub>**: Liters of component C1 used in the production of product GCA.\n*  **x<sub>2A</sub>**: Liters of component C2 used in the production of product GCA.\n*  **x<sub>1B</sub>**: Liters of component C1 used in the production of product GCB.\n*  **x<sub>2B</sub>**: Liters of component C2 used in the production of product GCB.\n*  **x<sub>1C</sub>**: Liters of component C1 used in the production of product GCC.\n*  **x<sub>2C</sub>**: Liters of component C2 used in the production of product GCC. \n'

In [133]:
response2.text

'Maximize: \n\n**120(x<sub>1A</sub> + x<sub>2A</sub>) + 135(x<sub>1B</sub> + x<sub>2B</sub>) + 155(x<sub>1C</sub> + x<sub>2C</sub>)** \n'

In [134]:
response3.text

'Subject to:\n\n**Availability Constraints:**\n\n*  x<sub>1A</sub> + x<sub>1B</sub> + x<sub>1C</sub> ≤ 10,000 (Liters of C1 available)\n*  x<sub>2A</sub> + x<sub>2B</sub> + x<sub>2C</sub> ≤ 15,000 (Liters of C2 available)\n\n**Demand Constraints:**\n\n*  x<sub>1A</sub> + x<sub>2A</sub> ≥ 6,000 (Demand for GCA)\n*  x<sub>1B</sub> + x<sub>2B</sub> ≥ 7,000 (Demand for GCB)\n*  x<sub>1C</sub> + x<sub>2C</sub> ≥ 9,000 (Demand for GCC)\n\n**Critical Element Constraints:**\n\n* 0.4x<sub>1A</sub> + 0.2x<sub>2A</sub> ≥ 0.3(x<sub>1A</sub> + x<sub>2A</sub>)  (Critical element in GCA)\n* 0.4x<sub>1B</sub> + 0.2x<sub>2B</sub> ≤ 0.3(x<sub>1B</sub> + x<sub>2B</sub>)  (Critical element in GCB) \n\n**Component Ratio Constraint:**\n\n* x<sub>1C</sub> ≥ 0.3x<sub>2C</sub> (Ratio of C1 to C2 in GCC)\n\n**Non-Negativity Constraints:**\n\n* x<sub>1A</sub>, x<sub>2A</sub>, x<sub>1B</sub>, x<sub>2B</sub>, x<sub>1C</sub>, x<sub>2C</sub> ≥ 0 \n'

In [135]:
response4.text

'```python\nimport pyomo.environ as pyo\n\n# Create a concrete model\nmodel = pyo.ConcreteModel()\n\n# Define decision variables\nmodel.x1A = pyo.Var(within=pyo.NonNegativeReals)\nmodel.x2A = pyo.Var(within=pyo.NonNegativeReals)\nmodel.x1B = pyo.Var(within=pyo.NonNegativeReals)\nmodel.x2B = pyo.Var(within=pyo.NonNegativeReals)\nmodel.x1C = pyo.Var(within=pyo.NonNegativeReals)\nmodel.x2C = pyo.Var(within=pyo.NonNegativeReals)\n\n# Define the objective function\nmodel.profit = pyo.Objective(\n    expr=120*(model.x1A + model.x2A) + 135*(model.x1B + model.x2B) + 155*(model.x1C + model.x2C), \n    sense=pyo.maximize\n)\n\n# Define constraints\nmodel.availability_C1 = pyo.Constraint(expr=model.x1A + model.x1B + model.x1C <= 10000)\nmodel.availability_C2 = pyo.Constraint(expr=model.x2A + model.x2B + model.x2C <= 15000)\n\nmodel.demand_GCA = pyo.Constraint(expr=model.x1A + model.x2A >= 6000)\nmodel.demand_GCB = pyo.Constraint(expr=model.x1B + model.x2B >= 7000)\nmodel.demand_GCC = pyo.Constrai